
# **TABLE TRANSFORMER - DETR**
Two functionalities implemented: Table Detection (TD) and Table Structure Recognition (TSR)

This notebook it is based in https://huggingface.co/spaces/SalML/TableTransformer2CSV/tree/main to obtain and postprocess the structure of the table obtained with table transformer from https://colab.research.google.com/drive/1lLRyBr7WraGdUJm-urUm_utArw6SkoCJ?usp=sharing. It is also based on https://github.com/microsoft/table-transformer 

## **1.- Dependencies**

In [1]:
!git clone https://huggingface.co/spaces/SalML/TableTransformer2CSV

Cloning into 'TableTransformer2CSV'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 53 (delta 23), reused 2 (delta 1), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [2]:
!pip install -r ./TableTransformer2CSV/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining transformers from git+https://github.com/nielsrogge/transformers.git@d34f7e6ffbb911d39465173ef2b35ba147ef58a9#egg=transformers (from -r ./TableTransformer2CSV/requirements.txt (line 45))
  Cloning https://github.com/nielsrogge/transformers.git (to revision d34f7e6ffbb911d39465173ef2b35ba147ef58a9) to ./src/transformers
  Running command git clone -q https://github.com/nielsrogge/transformers.git /content/src/transformers
  Running command git rev-parse -q --verify 'sha^d34f7e6ffbb911d39465173ef2b35ba147ef58a9'
  Running command git fetch -q https://github.com/nielsrogge/transformers.git d34f7e6ffbb911d39465173ef2b35ba147ef58a9
  Running command git checkout -q d34f7e6ffbb911d39465173ef2b35ba147ef58a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.1 MB 4.3 MB/s

Restart the enviroment!!


Check out the configuration reference at https://huggingface.co/docs/hub/spaces-config-reference



## **2.- Download dataset**


In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')

!cp "/content/drive/MyDrive/Colab Notebooks/oculist-sparkocr/dataset/oculist.zip" "/content/oculist.zip"
!unzip "/content/oculist.zip"


Mounted at /content/drive/
Archive:  /content/oculist.zip
   creating: images/
  inflating: images/1.jpeg           
  inflating: images/10.jpeg          
  inflating: images/11-1.jpg         
  inflating: images/11-2.jpg         
  inflating: images/12.jpeg          
  inflating: images/13.jpg           
  inflating: images/14.jpg           
  inflating: images/15.jpg           
  inflating: images/16.jpeg          
  inflating: images/17.jpg           
  inflating: images/18.jpg           
  inflating: images/19.jpg           
  inflating: images/2.jpeg           
  inflating: images/20.jpg           
  inflating: images/21.jpg           
  inflating: images/22.jpg           
  inflating: images/23.jpg           
  inflating: images/24-1.jpg         
  inflating: images/24-2.jpg         
  inflating: images/24-3.jpg         
  inflating: images/24-4.jpg         
  inflating: images/24-5.jpg         
  inflating: images/24.jpeg          
  inflating: images/3.jpg            
  inflati

## **3.- Inference**

In [2]:
#!python ./TableTransformer2CSV/app.py

2022-10-17 08:20:26.275 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
Traceback (most recent call last):
  File "./TableTransformer2CSV/app.py", line 18, in <module>
    from transformers import DetrFeatureExtractor
ModuleNotFoundError: No module named 'transformers'


In [4]:
import streamlit as st
from TableTransformer2CSV import *
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


st.set_option('deprecation.showPyplotGlobalUse', False)
st.set_page_config(layout='wide')
st.title("Table Detection and Table Structure Recognition")
st.write("Implemented by MSFT team: https://github.com/microsoft/table-transformer")


img_name = st.file_uploader("Upload an image with table(s)")

padd_top = st.slider('Padding top', 0, 200, 20)
padd_left = st.slider('Padding left', 0, 200, 20)
padd_right = st.slider('Padding right', 0, 200, 20)
padd_bottom = st.slider('Padding bottom', 0, 200, 20)


te = TableExtractionPipeline()
# for img in image_list:
if img_name is not None:
    asyncio.run(te.start_process(img_name, TD_THRESHOLD=0.6, TSR_THRESHOLD=0.8, padd_top=padd_top, padd_left=padd_left, padd_bottom=padd_bottom, padd_right=padd_right, delta_xmin=0, delta_ymin=0, delta_xmax=0, delta_ymax=0, expand_rowcol_bbox_top=0, expand_rowcol_bbox_bottom=0))




INFO:numexpr.utils:NumExpr defaulting to 2 threads.
2022-10-17 08:22:53.359 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2022-10-17 08:22:54.160 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


NameError: ignored

# TSR Evaluation

There are two different evaluation of TSR, with the dataset TrackB1 or with the TrackB2, as explained in:  https://github.com/cndplab-founder/ctdar_measurement_tool 

*   TRACK B1


In [ ]:
# Download dataset
!wget -O TrackB1_test_GT.zip https://zenodo.org/record/3239032/files/TrackB1_test_GT.zip?download=1
!unzip -d /content/ctdar_measurement_tool/annotations/ TrackB1_test_GT.zip

In [ ]:
# Remove images of dataset that are not in result, and remove result that there are not in dataset
processed = [get_fname(x) for x in glob.glob("/content/result/*")]
len(processed)
for f in glob.glob("/content/ctdar_measurement_tool/annotations/trackB1/*"):
    if get_fname(f) not in processed:
        os.remove(f)
dataset = [get_fname(x) for x in glob.glob("/content/ctdar_measurement_tool/annotations/trackB1/*")]
len(dataset)
for f in glob.glob("/content/result/*"):
    if get_fname(f) not in dataset:
        os.remove(f)

!tar -cvzf reg.tar.gz result/
!tar -cvzf res_img.tar.gz result_img/

In [ ]:
# Run evaluation
%cd "/content/ctdar_measurement_tool"
!python evaluate.py -trackB1 "/content/reg.tar.gz"
%cd "/content"

*   TRACK B2

In [ ]:
# Download dataset
!wget -O TrackB2_test_GT.zip https://zenodo.org/record/3239032/files/TrackB2_test_GT.zip?download=1
!unzip -d /content/ctdar_measurement_tool/annotations/ TrackB2_test_GT.zip

In [ ]:
# Remove images of dataset that are not in result, and remove result that there are not in dataset
processed = [get_fname(x) for x in glob.glob("/content/result/*")]
len(processed)
for f in glob.glob("/content/ctdar_measurement_tool/annotations/trackB2/*"):
    if get_fname(f) not in processed:
        os.remove(f)
dataset = [get_fname(x) for x in glob.glob("/content/ctdar_measurement_tool/annotations/trackB2/*")]
len(dataset)
for f in glob.glob("/content/result/*"):
    if get_fname(f) not in dataset:
        os.remove(f)

!tar -cvzf reg.tar.gz result/
!tar -cvzf res_img.tar.gz result_img/

In [ ]:
# Run evaluation
%cd "/content/ctdar_measurement_tool"
!python evaluate.py -trackB2 "/content/reg.tar.gz"
%cd "/content"